In [2]:
import pandas as pd
import seaborn as sns

# D.E. Project Write-Up

What if...

- **The data was increased by 100x.** Imagine if we had more cities with BIXI or different brand of bike sharing around the world. We could switch to a data Lake and work with Spark and S3. This would allow to store different types of format (e.g. different company have different types of logs). Furthermore, with more data it would be important to have a scalable structure with cloud services like AWS EMR which allow Spark to scale using multiple clusters.


- **The pipelines would be run on a daily basis by 7 am every day.** Imagine we would like to update each day the logs with the ones from yesterday. In this case, we could adapt our Apache Airflow pipeline to load new data every morning and compute our analyses for the day.


- **The database needed to be accessed by 100+ people.** Using AWS IAM (roles and users) works if we need the data to be access for more than a 100 people since it allows many users to work on the same data. Furthemore, we could use services such as  Amazon Cognito to manage user account. 

# D.E. Dictionnary

## Stations (dimension table)

| Variables  	| Description                                         	|
|------------	|-----------------------------------------------------	|
| station_id 	| Unique station ID   (PK)                            	|
| Longitude 	| Longitude coordinate                                	|
| Latitude   	| Latitude coordinate                                 	|
| Name       	| Official name of the station (streets intersection) 	|



## Daily Weather (dimension table)


| Variables     	| Description                                  	|
|---------------	|----------------------------------------------	|
| weather_id    	| Unique ID based on the date YYY-MM-DD (PK)	|
| temperature   	| The average temperature of the day (C°)      	|
| precipitation 	| The amount of precipitation for the day (mm) 	|


## Trips (fact table)

| Variables      	| Description                                                                	|
|----------------	|----------------------------------------------------------------------------	|
| trip_id        	| Unique trip id for each new trips (PK)                                     	|
| station_start  	| ID of the station where the bike was taken (trips start) (FK)              	|
| station_end    	| ID of the station where the bike was left (trip end) (FK)                  	|
| date_start     	| date-format (yyyy-mm-dd-hh-mm-ss) of the moment the bike ride started (FK) 	|
| date_end       	| date-format of the moment the bike ride ended (FK)                         	|
| weather_id     	| date-format (yyyy-mm-dd) ID to the daily weather (FK)                      	|
| trip_duration  	| lenght in seconds of the bike ride                                         	|
| is_member      	| if the user has a member pass (1) or not (0)                               	|

# Proof of working

To prove that the pipeline is working, I used the following query in Redshift to export the final data (trips for the example). Then, I used pandas to display the final tables (fact + dimensions).

**Redshift query:**

```
UNLOAD ('SELECT * FROM trips')   
TO 's3://bixi-project-udacity/redshift_trips' 
IAM_ROLE 'arn:aws:iam::XXXXXX:role/XXXXXXXX'
HEADER;
```

In [13]:
weather = pd.read_csv('./redshift_data/redshift_weather.csv')
weather.head()

,weather_id,temperature,precipitation
0,2020-01-01 00:00:00,0.2,0.9
1,2020-01-03 00:00:00,3.2,0.0
2,2020-01-05 00:00:00,-6.7,0.2
3,2020-01-07 00:00:00,-5.1,0.3
4,2020-01-09 00:00:00,-12.0,0.2


In [14]:
stations = pd.read_csv('./redshift_data/redshift_stations.csv')
stations.head()

,station_id,latitude,longitude,name
0,6001,45.510162,-73.556640,Métro Champ-de-Mars (Viger / Sanguinet)
1,6003,45.510822,-73.567170,Clark / Ontario
2,6005,45.500210,-73.571140,Metcalfe / du Square-Dorchester
3,6007,45.511660,-73.562130,de l'Hôtel-de-Ville / Ste-Catherine
4,6009,45.498110,-73.577614,Crescent / de Maisonneuve


In [25]:
trips = pd.read_csv('./redshift_data/redshift_trips_00.csv', sep='|')
trips.head()

,trip_id,station_start,station_end,date_start,date_end,weather_id,trip_duration,is_member
0,1,6100,6064,2018-04-10 11:56:00,2018-04-10 11:59:00,2018-04-10 00:00:00,122,t
1,5,6043,6041,2018-04-10 12:03:00,2018-04-10 12:05:00,2018-04-10 00:00:00,98,t
2,9,6137,6138,2018-04-10 12:06:00,2018-04-10 12:09:00,2018-04-10 00:00:00,161,t
3,13,6072,6012,2018-04-10 12:09:00,2018-04-10 12:17:00,2018-04-10 00:00:00,508,t
4,17,6073,6073,2018-04-10 12:12:00,2018-04-10 12:16:00,2018-04-10 00:00:00,272,t
